In [1]:
# This notebook illustrates handling the July 2024 Demo of the 6mo Hackathon Scenario 2, described at: 

SAVED_RESULTS_DIR = "./out"

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman_demo.example.pde import plot_spatial_timeseries
from IPython.display import HTML
import matplotlib.pyplot as plt
import itertools
from funman import FunmanWorkRequest, EncodingSchedule, POS_INFINITY

import json
from funman.representation.constraint import LinearConstraint, ParameterConstraint, StateVariableConstraint
from funman.representation import Interval
from funman import FUNMANConfig
import logging
from funman_demo import summarize_results



RESOURCES = "../resources"

EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet","mira")
MODEL_PATH = os.path.join(
    EXAMPLE_DIR, "models", "BIOMD0000000955_askenet.json"
)
REQUEST_PATH = os.path.join(
    EXAMPLE_DIR, "requests", "BIOMD0000000955_askenet_request.json"
)


# %load_ext autoreload
# %autoreload 2

In [ ]:


# N = "Total_population"
STATES = ["Susceptible", "Healed", "Infected", "Recognized", "Extinct", "Diagnosed", "Ailing", "Threatened"]
COMPARTMENTS = STATES
MAX_TIME=40
STEP_SIZE=10

with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))
    # funman_request.structure_parameters[0].schedules= []
    # for t in range(0, MAX_TIME, STEP_SIZE):
    #     funman_request.structure_parameters[0].schedules.append(EncodingSchedule(timepoints=list(range(0, t+STEP_SIZE, STEP_SIZE))))
    

    # # funman_request.config.substitute_subformulas = True
    # # funman_request.config.use_transition_symbols = True
    # funman_request.config.use_compartmental_constraints=True
    # funman_request.config.save_smtlib="./out"
    funman_request.config.tolerance = 0.01
#     funman_request.config.verbosity = 10
    # funman_request.config.dreal_log_level = "info"

    # factor = 1.6 
    to_synthesize = ["theta", "epsilon"]
    

    results_unconstrained_point = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE Hackathon 6mo",
            case_out_dir=SAVED_RESULTS_DIR,
            dump_plot=True,
            print_last_time=True,
            parameters_to_plot=to_synthesize
    )
    summary = summarize_results(COMPARTMENTS, results_unconstrained_point, ylabel="Population", parameters_to_plot=to_synthesize, synthesized_parameters=to_synthesize)
    print(summary)

In [2]:
# Demonstrate constructing a new AMR from a FUNMAN Result
from funman import FunmanResults
from functools import reduce
result_file = "out/09b4a613-6be1-4337-ad53-754e196609e1.json"
with open(result_file, "r") as f:
    results = FunmanResults.model_validate(json.load(f))

results.contract_model()

list(zip(results.model.petrinet.semantics.ode.parameters, results.contracted_model.semantics.ode.parameters))

Automatic initialization of gaol... done


[(Parameter(id='beta', name=None, description=None, value=0.011000000000000001, grounding=None, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.013200000000000002}), units=None),
  Parameter(id='beta', name=None, description=None, value=0.011000000000000001, grounding=None, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.013200000000000002}), units=None)),
 (Parameter(id='gamma', name=None, description=None, value=0.456, grounding=None, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.3648, 'maximum': 0.5472}), units=None),
  Parameter(id='gamma', name=None, description=None, value=0.456, grounding=None, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.3648, 'maximum': 0.5472}), units=None)),
 (Parameter(id='delta', name=None, description=None, value=0.011000000000000001, grounding=None, distribution=Distribution

In [ ]:
from funman import FunmanResults
out_file = REQUEST_PATH = os.path.join(
    "..", "out", "cbd1bbf2-e053-43a9-a587-d457fb0d577d.json"
)
with open(out_file, "r") as f:
    results = FunmanResults.model_validate(json.load(f))

In [ ]:
fp_boxes = (b for b in results.parameter_space.true_boxes if b.timestep().ub == 13)


In [ ]:
fp_box1 = fp_box

In [ ]:
fp_box.bounds["beta"].lb =  0.000001
fp_box.bounds["beta"].ub =  0.002
fp_box.bounds["gamma"].lb =  0.082
fp_box.bounds["gamma"].ub =  0.085

In [ ]:
from funman import FunmanWorkRequest


# fp_box = next(fp_boxes)

# print(fp_box.bounds)

with open("../resources/amr/petrinet/terrarium-tests/constrained_trajectory.json", "r") as request:
    r_dict = json.load(request)
#     funman_request = FunmanWorkRequest.model_validate(r_dict)
    # funman_request.structure_parameters[0].schedules= []
    # for t in range(0, MAX_TIME, STEP_SIZE):
    #     funman_request.structure_parameters[0].schedules.append(EncodingSchedule(timepoints=list(range(0, t+STEP_SIZE, STEP_SIZE))))
    

    # # funman_request.config.substitute_subformulas = True
    # # funman_request.config.use_transition_symbols = True
    # funman_request.config.use_compartmental_constraints=True
    # funman_request.config.save_smtlib="./out"
# results.request.config.tolerance = 0.01
# funman_request.config.verbosity = 20
# funman_request.config.dreal_log_level = "info"

# factor = 1.6 
to_synthesize = ["beta", "gamma"]
# beta = results.request.parameter("beta")
# beta.interval.lb = fp_box.bounds["beta"].lb
# beta.interval.ub = fp_box.bounds["beta"].ub

# gamma = results.request.parameter("gamma")
# gamma.interval.lb = fp_box.bounds["gamma"].lb
# gamma.interval.ub = fp_box.bounds["gamma"].ub
    
    # keep_constant = ["p_Vax_UnVax", "p_UnVax_Vax"]
    # for p in [po for po in funman_request.parameters if po.name not in keep_constant # if po.name  in to_synthesize
    # ]:
    #     param = funman_request.parameter(p.name)
    #     param.interval.lb = max(p.interval.lb-factor, 0)
    #     param.interval.ub = min(p.interval.ub+factor, max(1, p.interval.ub+factor))
    #     # param.label = "all" if p.name in to_synthesize else "any"
    #     param.label = "any"
    #     print(param)

    # # Susceptible is decreasing,  dS/dt <= 0, S_{t+1} - S_t <= 0
    # c1 = LinearConstraint(name="c1", additive_bounds=Interval(ub=0, closed_upper_bounds=True), variables=["Susceptible"], derivative=True)
    # funman_request.constraints.append(c1)

    # # # R is increasing, dR/dt >= 0,   R_{t+1} - R_t >= 0
    # c2 = LinearConstraint(name="c2", additive_bounds=Interval(lb=0), variables=["Recovered"], derivative=True)
    # funman_request.constraints.append(c2)

    # # # D is increasing, dD/dt >= 0, dD/dt >= 0,   D_{t+1} - D_t >= 0
    # c3 = LinearConstraint(name="c3", additive_bounds=Interval(lb=0), variables=["Deceased"], derivative=True)
    # funman_request.constraints.append(c3)

    # # # Population size maintained
    # # c4 = LinearConstraint(name="compartment", additive_bounds=Interval(lb=0, ub=0, closed_upper_bound=True), variables=COMPARTMENTS, weights=[1, 1, 1, 1, 1, -1])
    # # funman_request.constraints.append(c4)

    # # # Infected peak
    # # c5 = LinearConstraint(name="Infected_peak", additive_bounds=Interval(lb=50), variables=["Infected"], timepoints=Interval(lb=40, ub=40, closed_upper_bound=True))
    # # funman_request.constraints.append(c5)

    # # for sv in STATES:
    # #     funman_request.constraints.append(StateVariableConstraint(name=f"{sv}_bounds", variable=sv, interval=Interval(lb=0, ub=100, closed_upper_bound=True)))
    

    


results_unconstrained_point = Runner().run(
            r_dict["model"],
            # results.request,
            r_dict["request"],
            description="SIDARTHE Hackathon 6mo",
            case_out_dir=SAVED_RESULTS_DIR,
            dump_plot=True,
            print_last_time=True,
            parameters_to_plot=to_synthesize
    )
    # summary = summarize_results(COMPARTMENTS, results_unconstrained_point, ylabel="Population", parameters_to_plot=to_synthesize+["timestep"], synthesized_parameters=to_synthesize+["timestep"])
    # print(summary)